In [37]:
!pip install crewai groq python-dotenv pandas chromadb
!pip install kaggle

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["GROQ_API_KEY"] = "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"

In [39]:
import os

os.environ['KAGGLE_USERNAME'] = "yashdoshi8"
os.environ['KAGGLE_KEY'] = "52345c4fd7732ce6cb4c3218d4a1ce88"

In [40]:
!kaggle datasets download -d rockbottom73/leads-dataset
!kaggle datasets download -d kshitizregmi/jobs-and-job-description
!kaggle datasets download -d gauravduttakiit/resume-dataset

Dataset URL: https://www.kaggle.com/datasets/rockbottom73/leads-dataset
License(s): unknown
leads-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/kshitizregmi/jobs-and-job-description
License(s): CC0-1.0
jobs-and-job-description.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/gauravduttakiit/resume-dataset
License(s): CC0-1.0
resume-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [41]:
import zipfile

with zipfile.ZipFile("leads-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset1")
with zipfile.ZipFile("jobs-and-job-description.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset2")
with zipfile.ZipFile("resume-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset3")

In [42]:
import pandas as pd

leads_df = pd.read_csv("/content/dataset1/SampleData.csv", encoding="latin1")
leads_df.head()

/tmp/ipython-input-1412392318.py:3: DtypeWarning: Columns (5,44,53,77,97) have mixed types. Specify dtype option on import or set low_memory=False.
  leads_df = pd.read_csv("/content/dataset1/SampleData.csv", encoding="latin1")


,Prospect ID,Lead Number,Company,Lead Origin,Mobile Number,Website,Time Zone,Job Title,Lead Source,Source Medium,...,Stage Rotting Flag Level,Mailing Preferences,Twitter Id,Facebook Id,LinkedIn Id,Skype Id,Gtalk Id,Google Plus Id,Quality Score 01,Groups
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,NaN,API,NaN,NaN,NaN,NaN,Olark Chat,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,NaN,API,NaN,NaN,NaN,NaN,Organic Search,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,NaN,Landing Page Submission,NaN,NaN,NaN,NaN,Direct Traffic,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,NaN,Landing Page Submission,NaN,NaN,NaN,NaN,Direct Traffic,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3256f628-e534-4826-9d63-4a8b88782852,660681,NaN,Landing Page Submission,NaN,NaN,NaN,NaN,Google,CPC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
jobs_df = pd.read_csv("/content/dataset2/job_title_des.csv")
jobs_df.head()

,Unnamed: 0,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...
4,4,Full Stack Developer,job responsibility full stack engineer – react...


In [44]:
try:
    resume_df = pd.read_csv("/content/dataset3/UpdatedResumeDataSet.csv")
    resume_df.head()
    use_resume = True
except:
    print("No resume dataset found. Skipping optional layer.")
    use_resume = False


In [48]:
from groq import Groq
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
import os

load_dotenv()

client = Groq(api_key=os.environ["GROQ_API_KEY"])

from crewai import LLM

groq_llm = LLM(
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

In [49]:
#AGENT

lead_enrichment_agent = Agent(
    name="Lead Enrichment Agent",
    role="Data Enrichment Specialist",
    goal="Enhance lead data using synthetic reasoning.",
    backstory="Fills missing profile fields for personalization.",
    llm=groq_llm
)

job_analysis_agent = Agent(
    name="Job Analysis Agent",
    role="JD Breakdown Specialist",
    goal="Extract key job qualifications & themes.",
    backstory="Understands what hiring managers look for.",
    llm=groq_llm
)

email_writer_agent = Agent(
    name="Email Writer Agent",
    role="Personalized Outreach Expert",
    goal="Write targeted outreach emails.",
    backstory="Writes personalized job emails for maximum response.",
    llm=groq_llm
)

email_quality_agent = Agent(
    name="Email Quality Agent",
    role="Email Scoring Specialist",
    goal="Score emails for clarity, tone, and personalization.",
    backstory="Expert quality evaluator.",
    llm=groq_llm
)

email_scorer_agent = Agent(
    name="Email Scoring Agent",
    role="Email Quality Evaluator",
    goal="Analyze outreach email quality and provide a structured score.",
    backstory=(
        "Expert in outreach communication patterns, personalization techniques, "
        "and persuasive writing frameworks."
    ),
    llm=groq_llm
)


In [50]:
#TASK

# 1. LEAD ENRICHMENT
lead_enrichment_task = Task(
    description=(
        "You are given raw lead data:\n\n"
        "{{lead_data}}\n\n"
        "Extract and enrich this into:\n"
        "- name\n"
        "- email\n"
        "- company\n"
        "- role / seniority\n"
        "- short bio\n\n"
        "Return ONLY valid JSON with keys: name, email, company, role, bio."
    ),
    expected_output="A JSON dictionary with keys: name, email, company, role, bio.",
    agent=lead_enrichment_agent
)

# 2. JOB ANALYSIS
job_analysis_task = Task(
    description=(
        "Analyze this job posting:\n\n"
        "{{job_data}}\n\n"
        "Extract and summarize:\n"
        "- job_title\n"
        "- company\n"
        "- top_skills_requested\n"
        "- responsibilities_summary\n\n"
        "Return ONLY valid JSON."
    ),
    expected_output="A JSON dictionary with job_title, company, top_skills_requested, responsibilities_summary.",
    agent=job_analysis_agent
)

# 3. EMAIL GENERATION
email_generation_task = Task(
    description=(
        "Write a personalized job outreach email using:\n"
        "- persona from lead enrichment\n"
        "- job analysis output\n"
        "Tone to use: {{tone}}\n\n"
        "Generate:\n"
        "- subject line\n"
        "- email body\n\n"
        "Use only context passed from previous tasks."
    ),
    expected_output="A dictionary with keys: subject_line and email_body.",
    agent=email_writer_agent,
    context=[lead_enrichment_task, job_analysis_task]
)

# 4. EMAIL SCORING
email_scoring_task = Task(
    description=(
        "Evaluate the generated outreach email on:\n"
        "- personalization\n"
        "- tone match\n"
        "- clarity\n"
        "- professionalism\n"
        "Score 1–10 and provide a brief explanation."
    ),
    expected_output="A dictionary containing score (1–10) and explanation.",
    agent=email_scorer_agent,
    context=[email_generation_task]
)


In [51]:
email_crewai = Crew(
    agents=[
        lead_enrichment_agent,
        job_analysis_agent,
        email_writer_agent,
        email_scorer_agent
    ],
    tasks=[
        lead_enrichment_task,
        job_analysis_task,
        email_generation_task,
        email_scoring_task
    ],
    verbose=True
)

In [52]:
def run_pipeline(lead_row, job_row, tone="formal"):

    inputs = {
        "lead_data": lead_row,
        "job_data": job_row,
        "tone": tone
    }

    result = email_crewai.kickoff(inputs=inputs)

    return result

In [57]:
sample_lead = leads_df.iloc[0].to_dict()
sample_job = jobs_df.iloc[0].to_dict()

result = run_pipeline(sample_lead, sample_job, tone="persuasive")

print(result.raw)      # only LLM output
print(result.json_dict)  # if the task returned JSON


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'Prospect ID': '7927b2df-8bba-4d29-b9a2-b6e0beafe620', 'Lead Number': 660737, 'Company': nan, 'Lead          │
│  Origin': 'API', 'Mobile Number': nan, 'Website': nan, 'Time Zone': nan, 'Job Title': nan, 'Lead Source':       │
│  'Olark Chat', 'Source Medium': nan, 'Notes': nan, 'Source Campaign': nan, 'Source Content': nan, 'Do Not       │
│  Email': 'No', 'Do Not Call': 'No', 'Lead Stage': 'Not Interested', 'Lead Grade': nan, 'Lead Score': 105,       │
│  'Order Value': nan, 'Engagement Score': 0, 'TotalVisits': 0.0, 'Page Views Per Visit': 0.0, 'Average Time Per  │
│  Visit': 0.0, 'Last Activity': 'Page Visited on Website', 'Last Activity Date': '31-07-2016 17:16', 'Related    │
│  Landing Page Id': nan, 'First Landing Page Submission Id': nan, 'First Landing Page Submission Date': nan,     │
│  'Created On': '31-07-2016 17:11', 'Modified On': '03-08-2016 12:00', 'Lead Conversion Date': nan, 'Address     │
│  1': nan, 'Address 2': nan, 'Cityold': nan, 'State': nan, 'Country': nan, 'Zip': nan, 'Facebook URL': nan,      │
│  'Twitter URL': nan, 'LinkedIn URL': nan, 'Industry': nan, 'Work Area': nan, 'Course Interested': nan,          │
│  'Keyword': nan, 'Date': nan, 'Specialization': 'Select Specialization', 'Entrance Test': 'MAT', 'How did you   │
│  hear about SomeSchool': 'Select', 'What is your current occupation': 'Unemployed', 'If you are a working       │
│  professional': 'Accounting/Finance', 'If you are a working professional please mention ': 'Select', 'What      │
│  matters most to you in choosing an ADP': 'Better Career Prospects', 'Age': 'Select', 'Next Follow Up': nan,    │
│  'Search': 'No', 'Magazine': 'No', 'Newspaper Article': 'No', 'Welearn Forums': 'No', 'Newspaper': 'No',        │
│  'Digital Advertisement': 'No', 'Through Recommendations': 'No', 'Any other Please specify': nan, 'Last         │
│  Degree': nan, 'Receive More Updates About Our Courses': 'No', 'Admission Number': nan, 'Tags': 'Interested in  │
│  other courses', 'Lead Quality': 'Low in Relevance', 'Update me on Supply Chain Content': 'No', 'Get updates    │
│  on PGDMHBSCM': 'No', 'Chat Group': nan, 'Lead Profile': 'Select', 'City New': 'Select', 'Country New':         │
│  'India', 'Industry New': 'Consumer Durables', 'Profile Score': nan, 'Asymmetrique Activity Index':             │
│  '02.Medium', 'Asymmetrique Profile Index': '02.Medium', 'ecode': nan, 'amt': nan, 'eventname': nan, 'Enquiry   │
│  Type': nan, 'Asymmetrique Activity Score': 15.0, 'Asymmetrique Profile Score': 15.0, 'Admission Date': nan,    │
│  'I agree to pay the amount through cheque': 'No', 'Previous Stage': 'Select', 'Number of Followup Calls':      │
│  'Select', 'Asym how soon you can join program': 'Select', 'Asymm Reason to take admission': 'Select', 'Asymm   │
│  Are you aware about SomeSchool College': 'Select', 'Asymm Are you applying for any other form of MBA':         │
│  'Select', 'Asym What is your expectation from this course': 'Select', 'Asym How do you feel about taking       │
│  admission': 'Select', 'Asym What stopping you from joining course': 'Select', 'Asym What are your concerns     │
│  for taking admission': 'Select', 'Asym preferences for selecting management course': 'Select', 'a free copy    │
│  of Mastering The Interview': 'No', 'What attracted you

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│      "name": null,                                                                                              │
│      "email": null,                                                                                             │
│      "company": null,                                                                                           │
│      "role": "Unemployed, previously in Accounting/Finance",                                                    │
│      "bio": " Interested in other courses, Low in Relevance lead quality, searching for better career           │
│  prospects, previously worked in Consumer Durables industry."                                                   │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│      "score": 6,                                                                                                │
│      "explanation": "The email has some personalized elements, such as addressing the hiring manager and        │
│  expressing interest in the specific Flutter Developer position. However, the tone could be more                │
│  conversational and less formal, and the clarity of the email could be improved by more clearly stating the     │
│  candidate's relevant skills and qualifications. The professionalism of the email is good, with proper          │
│  greetings and sign-offs. Overall, the email is well-structured but could benefit from more specific examples   │
│  and a stronger demonstration of the candidate's passion for the tech industry and Flutter development.         │
│  Additionally, the email could be improved by removing the mention of the job offer benefits, as it may come    │
│  across as overly focused on the perks rather than the candidate's qualifications and fit for the role."        │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

None


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│      "score": 6,                                                                                                │
│      "explanation": "The email has some personalized elements, such as addressing the hiring manager and        │
│  expressing interest in the specific Flutter Developer position. However, the tone could be more                │
│  conversational and less formal, and the clarity of the email could be improved by more clearly stating the     │
│  candidate's relevant skills and qualifications. The professionalism of the email is good, with proper          │
│  greetings and sign-offs. Overall, the email is well-structured but could benefit from more specific examples   │
│  and a stronger demonstration of the candidate's passion for the tech industry and Flutter development.         │
│  Additionally, the email could be improved by removing the mention of the job offer benefits, as it may come    │
│  across as overly focused on the perks rather than the candidate's qualifications and fit for the role."        │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [54]:
os.environ["CREWAI_ENABLE_TRACE"] = "false"
os.environ["CREWAI_DISABLE_TELEMETRY"] = "true"

In [77]:
test_lead_1 = {
    "First Name": "Alicia",
    "Last Name": "Raymond",
    "Email": "alicia.raymond@example.com",
    "Job Title": "Data Analyst",
    "Company": "NielsenIQ",
    "Experience": "3 years in data analytics",
}

test_job_1 = {
    "Job Title": "Senior Data Analyst",
    "Company": "Meta",
    "Description": "Looking for an analyst with SQL, Python, dashboards, experimentation frameworks."
}

def run_pipeline(lead_row, job_row, tone="formal"):
    inputs = {
        "lead_data": lead_row,   # must match your task template
        "job_data": job_row,     # must match your task template
        "tone": tone
    }

    result = email_crewai.kickoff(inputs)
    return result

result = run_pipeline(test_lead_1, test_job_1, tone="persuasive")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'First Name': 'Alicia', 'Last Name': 'Raymond', 'Email': 'alicia.raymond@example.com', 'Job Title': 'Data    │
│  Analyst', 'Company': 'NielsenIQ', 'Experience': '3 years in data analytics'}}                                  │
│                                                                                                                 │
│  Extract and enrich this into:                                                                                  │
│  - name                                                                                                         │
│  - email                                                                                                        │
│  - company                                                                                                      │
│  - role / seniority                                                                                             │
│  - short bio                                                                                                    │
│                                                                                                                 │
│  Return ONLY valid JSON with keys: name, email, company, role, bio.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"name": "Alicia Raymond", "email": "alicia.raymond@example.com", "company": "NielsenIQ", "role": "Data        │
│  Analyst", "bio": "Data Analyst with 3 years of experience in data analytics"}                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Task: Evaluate the generated outreach email on:                                                                │
│  - personalization                                                                                              │
│  - tone match                                                                                                   │
│  - clarity                                                                                                      │
│  - professionalism                                                                                              │
│  Score 1–10 and provide a brief explanation.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│  "score": 9,                                                                                                    │
│  "explanation": "The outreach email scores high in personalization as it addresses the recipient by name        │
│  (Alicia Raymond) and references their specific experience and background. The tone is professional and         │
│  matches the formal context of a job opportunity. The email is clear in its intention and provides a concise    │
│  overview of the role and its responsibilities. The language used is professional, and the structure of the     │
│  email is logical and easy to follow. The only area for improvement could be adding more specific details       │
│  about the company culture and what makes this role unique, but overall, the email effectively conveys the      │
│  opportunity and invites the recipient to discuss further."                                                     │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│  "score": 9,                                                                                                    │
│  "explanation": "The outreach email scores high in personalization as it addresses the recipient by name        │
│  (Alicia Raymond) and references their specific experience and background. The tone is professional and         │
│  matches the formal context of a job opportunity. The email is clear in its intention and provides a concise    │
│  overview of the role and its responsibilities. The language used is professional, and the structure of the     │
│  email is logical and easy to follow. The only area for improvement could be adding more specific details       │
│  about the company culture and what makes this role unique, but overall, the email effectively conveys the      │
│  opportunity and invites the recipient to discuss further."                                                     │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [78]:
test_lead_2 = {
    "name": "Ethan Parker",
    "company": "AppForge Labs",
    "notes": "Looking for mobile dev who cares about UX and animations."
}

test_job_2 = {
    "title": "Flutter Developer",
    "skills": "Flutter, Dart, animations, REST APIs",
    "description": "Help build a highly interactive mobile learning platform."
}

def run_pipeline(lead_row, job_row, tone="formal"):
    inputs = {
        "lead_data": lead_row,   # must match your task template
        "job_data": job_row,     # must match your task template
        "tone": tone
    }

    result = email_crewai.kickoff(inputs)
    return result

result = run_pipeline(test_lead_2, test_job_2, tone="friendly")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'name': 'Ethan Parker', 'company': 'AppForge Labs', 'notes': 'Looking for mobile dev who cares about UX and  │
│  animations.'}}                                                                                                 │
│                                                                                                                 │
│  Extract and enrich this into:                                                                                  │
│  - name                                                                                                         │
│  - email                                                                                                        │
│  - company                                                                                                      │
│  - role / seniority                                                                                             │
│  - short bio                                                                                                    │
│                                                                                                                 │
│  Return ONLY valid JSON with keys: name, email, company, role, bio.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"name": "Ethan Parker", "email": "ethan.parker@appforgelabs.com", "company": "AppForge Labs", "role":         │
│  "Mobile Development Lead", "bio": "Mobile development expert with a focus on UX and animations, leading        │
│  projects at AppForge Labs"}                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Task: Evaluate the generated outreach email on:                                                                │
│  - personalization                                                                                              │
│  - tone match                                                                                                   │
│  - clarity                                                                                                      │
│  - professionalism                                                                                              │
│  Score 1–10 and provide a brief explanation.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│  "score": 9,                                                                                                    │
│  "explanation": "The outreach email demonstrates strong personalization, addressing the recipient by name and   │
│  referencing their specific experience and expertise. The tone is professional and courteous, with a match      │
│  between the tone and the purpose of the email. The email is clear and concise, effectively conveying the       │
│  opportunity and its relevance to the recipient's skills and interests. The only area for improvement could be  │
│  adding more specific details about the company and the role, but overall, the email is well-structured and     │
│  shows a genuine interest in the recipient's potential fit for the position."                                   │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│  "score": 9,                                                                                                    │
│  "explanation": "The outreach email demonstrates strong personalization, addressing the recipient by name and   │
│  referencing their specific experience and expertise. The tone is professional and courteous, with a match      │
│  between the tone and the purpose of the email. The email is clear and concise, effectively conveying the       │
│  opportunity and its relevance to the recipient's skills and interests. The only area for improvement could be  │
│  adding more specific details about the company and the role, but overall, the email is well-structured and     │
│  shows a genuine interest in the recipient's potential fit for the position."                                   │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [79]:
test_lead_3 = {
    "name": "Sarah Carter",
    "company": "American Express",
    "notes": "Cares about campaign performance + segmentation."
}

test_job_3 = {
    "title": "Marketing Analytics Intern",
    "skills": "SQL, Tableau, customer segmentation",
    "description": "Analyze cardmember behavior and optimize acquisition funnels."
}
def run_pipeline(lead_row, job_row, tone="formal"):
    inputs = {
        "lead_data": lead_row,   # must match your task template
        "job_data": job_row,     # must match your task template
        "tone": tone
    }

    result = email_crewai.kickoff(inputs)
    return result

result = run_pipeline(test_lead_3, test_job_3, tone="friendly")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'name': 'Sarah Carter', 'company': 'American Express', 'notes': 'Cares about campaign performance +          │
│  segmentation.'}}                                                                                               │
│                                                                                                                 │
│  Extract and enrich this into:                                                                                  │
│  - name                                                                                                         │
│  - email                                                                                                        │
│  - company                                                                                                      │
│  - role / seniority                                                                                             │
│  - short bio                                                                                                    │
│                                                                                                                 │
│  Return ONLY valid JSON with keys: name, email, company, role, bio.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"name": "Sarah Carter", "email": "sarah.carter@americanexpress.com", "company": "American Express", "role":   │
│  "Marketing Manager", "bio": "Sarah Carter is a marketing professional at American Express, with a focus on     │
│  campaign performance and segmentation. She has a strong interest in data-driven decision making and is likely  │
│  responsible for overseeing marketing campaigns and analyzing their effectiveness."}                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Task: Evaluate the generated outreach email on:                                                                │
│  - personalization                                                                                              │
│  - tone match                                                                                                   │
│  - clarity                                                                                                      │
│  - professionalism                                                                                              │
│  Score 1–10 and provide a brief explanation.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c27ab158-847e-4d9f-8023-e911bb2c14f7                                                                     │
│  Agent: Personalized Outreach Expert                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [80]:
test_lead_4 = {
    "name": "Daniel Morris",
    "company": "Google",
    "notes": "Interested in backend engineers with distributed systems exposure."
}

test_job_4 = {
    "title": "Software Engineer Intern - Backend",
    "skills": "Python, Go, distributed systems, microservices",
    "description": "Work with high-scale backend services powering Google Search."
}
def run_pipeline(lead_row, job_row, tone="formal"):
    inputs = {
        "lead_data": lead_row,   # must match your task template
        "job_data": job_row,     # must match your task template
        "tone": tone
    }

    result = email_crewai.kickoff(inputs)
    return result

result = run_pipeline(test_lead_4, test_job_4, tone="friendly")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'name': 'Daniel Morris', 'company': 'Google', 'notes': 'Interested in backend engineers with distributed     │
│  systems exposure.'}}                                                                                           │
│                                                                                                                 │
│  Extract and enrich this into:                                                                                  │
│  - name                                                                                                         │
│  - email                                                                                                        │
│  - company                                                                                                      │
│  - role / seniority                                                                                             │
│  - short bio                                                                                                    │
│                                                                                                                 │
│  Return ONLY valid JSON with keys: name, email, company, role, bio.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"name": "Daniel Morris", "email": "daniel.morris@google.com", "company": "Google", "role": "Hiring            │
│  Manager/Engineering Lead", "bio": "Experienced engineering lead at Google, specializing in distributed         │
│  systems and backend engineering, with a focus on recruiting top talent in the field."}                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Task: Evaluate the generated outreach email on:                                                                │
│  - personalization                                                                                              │
│  - tone match                                                                                                   │
│  - clarity                                                                                                      │
│  - professionalism                                                                                              │
│  Score 1–10 and provide a brief explanation.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c27ab158-847e-4d9f-8023-e911bb2c14f7                                                                     │
│  Agent: Personalized Outreach Expert                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│  "score": 8,                                                                                                    │
│  "explanation": "The email is well-structured and personalized, addressing the recipient by name (Daniel) and   │
│  referencing a specific role (Software Engineer Intern - Backend) that aligns with their potential interests.   │
│  The tone is professional and friendly, with a clear call to action (a brief call to discuss further).          │
│  However, the email could be improved with more specific details about how the sender's services or             │
│  connections can benefit the recipient, and why they are the best fit for this particular role. Additionally,   │
│  the email assumes the recipient is interested in the role without prior confirmation, which might not always   │
│  be the case. Overall, the email demonstrates good personalization, tone match, clarity, and professionalism,   │
│  but could benefit from more tailored content and a clearer value proposition."                                 │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [81]:
test_lead_5 = {
    "name": "Priya Narayanan",
    "company": "Target",
    "notes": "Wants people who improve data pipelines and reporting latency."
}

test_job_5 = {
    "title": "Data Engineering Intern",
    "skills": "Spark, Python, ETL, cloud data warehouses",
    "description": "Optimize inventory and forecasting pipelines."
}
def run_pipeline(lead_row, job_row, tone="formal"):
    inputs = {
        "lead_data": lead_row,   # must match your task template
        "job_data": job_row,     # must match your task template
        "tone": tone
    }

    result = email_crewai.kickoff(inputs)
    return result

result = run_pipeline(test_lead_5, test_job_5, tone="friendly")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'name': 'Priya Narayanan', 'company': 'Target', 'notes': 'Wants people who improve data pipelines and        │
│  reporting latency.'}}                                                                                          │
│                                                                                                                 │
│  Extract and enrich this into:                                                                                  │
│  - name                                                                                                         │
│  - email                                                                                                        │
│  - company                                                                                                      │
│  - role / seniority                                                                                             │
│  - short bio                                                                                                    │
│                                                                                                                 │
│  Return ONLY valid JSON with keys: name, email, company, role, bio.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│  "name": "Priya Narayanan",                                                                                     │
│  "email": "priya.narayanan@target.com",                                                                         │
│  "company": "Target",                                                                                           │
│  "role": "Data Engineering Lead",                                                                               │
│  "bio": "Priya Narayanan is a data engineering lead at Target, focused on improving data pipelines and          │
│  reducing reporting latency to drive business growth."                                                          │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Task: Evaluate the generated outreach email on:                                                                │
│  - personalization                                                                                              │
│  - tone match                                                                                                   │
│  - clarity                                                                                                      │
│  - professionalism                                                                                              │
│  Score 1–10 and provide a brief explanation.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c27ab158-847e-4d9f-8023-e911bb2c14f7                                                                     │
│  Agent: Personalized Outreach Expert                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│  "score": 8,                                                                                                    │
│  "explanation": "The email scores high on personalization as it addresses the recipient by name (Priya) and     │
│  references their specific work and company (Target), showing that the sender has taken the time to research    │
│  the recipient's background. The tone is professional and polite, making it a good match for a business email.  │
│  The clarity of the email is also strong, as the purpose of the email (exploring opportunities for a Data       │
│  Engineering Intern) is clearly stated and the requirements for the role are specified. However, the email      │
│  could be improved by providing more detail about the company and the benefits of the internship, which would   │
│  help to make the opportunity more attractive to the recipient. Additionally, the email assumes that the        │
│  recipient is interested in hiring or knows someone who is, which might not always be the case. Minor           │
│  adjustments to make the email more engaging and less assumptive would elevate its quality."                    │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

{
"score": 8,
"explanation": "The email scores high on personalization as it addresses the recipient by name (Priya) and references their specific work and company (Target), showing that the sender has taken the time to research the recipient's background. The tone is professional and polite, making it a good match for a business email. The clarity of the email is also strong, as the purpose of the email (exploring opportunities for a Data Engineering Intern) is clearly stated and the requirements for the role are specified. However, the email could be improved by providing more detail about the company and the benefits of the internship, which would help to make the opportunity more attractive to the recipient. Additionally, the email assumes that the recipient is interested in hiring or knows someone who is, which might not always be the case. Minor adjustments to make the email more engaging and less assumptive would elevate its quality."
}


In [82]:
test_lead_6 = {
    "name": "Tom Richards",
    "company": "NVIDIA",
    "notes": "Focuses on LLM optimization + model compression."
}

test_job_6 = {
    "title": "AI Research Intern",
    "skills": "PyTorch, model optimization, quantization, LLMs",
    "description": "Work on compute-efficient model deployment."
}
def run_pipeline(lead_row, job_row, tone="formal"):
    inputs = {
        "lead_data": lead_row,   # must match your task template
        "job_data": job_row,     # must match your task template
        "tone": tone
    }

    result = email_crewai.kickoff(inputs)
    return result

result = run_pipeline(test_lead_6, test_job_6, tone="friendly")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'name': 'Tom Richards', 'company': 'NVIDIA', 'notes': 'Focuses on LLM optimization + model compression.'}}   │
│                                                                                                                 │
│  Extract and enrich this into:                                                                                  │
│  - name                                                                                                         │
│  - email                                                                                                        │
│  - company                                                                                                      │
│  - role / seniority                                                                                             │
│  - short bio                                                                                                    │
│                                                                                                                 │
│  Return ONLY valid JSON with keys: name, email, company, role, bio.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│      "name": "Tom Richards",                                                                                    │
│      "email": "tom.richards@nvidia.com",                                                                        │
│      "company": "NVIDIA",                                                                                       │
│      "role": "LLM Optimization Engineer",                                                                       │
│      "bio": "Tom Richards is an expert in LLM optimization and model compression at NVIDIA, focusing on         │
│  innovative AI solutions."                                                                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c27ab158-847e-4d9f-8023-e911bb2c14f7                                                                     │
│  Agent: Personalized Outreach Expert                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Task: Evaluate the generated outreach email on:                                                                │
│  - personalization                                                                                              │
│  - tone match                                                                                                   │
│  - clarity                                                                                                      │
│  - professionalism                                                                                              │
│  Score 1–10 and provide a brief explanation.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│      "score": 9,                                                                                                │
│      "explanation": "The email scores high on personalization as it addresses the recipient by name ('Hi Tom')  │
│  and shows familiarity with their work and expertise ('I came across your profile and was impressed by your     │
│  work in LLM optimization and model compression at NVIDIA'). The tone is professional and inviting, matching    │
│  the formal context of a job opportunity discussion. The clarity of the email is also commendable, as it        │
│  clearly states the purpose of the outreach ('I wanted to reach out to you regarding the AI Research Intern     │
│  position at NVIDIA') and outlines the key requirements of the role ('The role requires proficiency in          │
│  PyTorch, model optimization, quantization, and LLMs'). The professionalism of the email is evident in its      │
│  polished language, proper structure, and respectful closing ('Best regards, [Your Name]'). The only aspect     │
│  that prevents a perfect score is the lack of a direct call to action or specific next steps for the recipient  │
│  to take, which would further enhance the email's effectiveness."                                               │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{
    "score": 9,
    "explanation": "The email scores high on personalization as it addresses the recipient by name ('Hi Tom') and shows familiarity with their work and expertise ('I came across your profile and was impressed by your work in LLM optimization and model compression at NVIDIA'). The tone is professional and inviting, matching the formal context of a job opportunity discussion. The clarity of the email is also commendable, as it clearly states the purpose of the outreach ('I wanted to reach out to you regarding the AI Research Intern position at NVIDIA') and outlines the key requirements of the role ('The role requires proficiency in PyTorch, model optimization, quantization, and LLMs'). The professionalism of the email is evident in its polished language, proper structure, and respectful closing ('Best regards, [Your Name]'). The only aspect that prevents a perfect score is the lack of a direct call to action or specific next steps for the recipient to take, which would 

In [83]:
test_lead_7 = {
    "name": "Olivia Chen",
    "company": "Two Sigma",
    "notes": "Prefers statistical modeling backgrounds."
}

test_job_7 = {
    "title": "Quantitative Research Intern",
    "skills": "Python, statistics, ML, time-series analysis",
    "description": "Develop predictive trading signals."
}

def run_pipeline(lead_row, job_row, tone="formal"):
    inputs = {
        "lead_data": lead_row,   # must match your task template
        "job_data": job_row,     # must match your task template
        "tone": tone
    }

    result = email_crewai.kickoff(inputs)
    return result

result = run_pipeline(test_lead_7, test_job_7, tone="friendly")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ddb6eef3-8391-42c7-82a1-47f1516d50b3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Task: You are given raw lead data:                                                                             │
│                                                                                                                 │
│  {{'name': 'Olivia Chen', 'company': 'Two Sigma', 'notes': 'Prefers statistical modeling backgrounds.'}}        │
│                                                                                                                 │
│  Extract and enrich this into:                                                                                  │
│  - name                                                                                                         │
│  - email                                                                                                        │
│  - company                                                                                                      │
│  - role / seniority                                                                                             │
│  - short bio                                                                                                    │
│                                                                                                                 │
│  Return ONLY valid JSON with keys: name, email, company, role, bio.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Enrichment Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"name": "Olivia Chen", "email": "olivia.chen@twosigma.com", "company": "Two Sigma", "role": "Quantitative     │
│  Analyst", "bio": "Olivia is a quantitative analyst with a strong background in statistical modeling, working   │
│  at Two Sigma, a renowned hedge fund."}                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Task: Evaluate the generated outreach email on:                                                                │
│  - personalization                                                                                              │
│  - tone match                                                                                                   │
│  - clarity                                                                                                      │
│  - professionalism                                                                                              │
│  Score 1–10 and provide a brief explanation.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c27ab158-847e-4d9f-8023-e911bb2c14f7                                                                     │
│  Agent: Personalized Outreach Expert                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Quality Evaluator                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│  "score": 8,                                                                                                    │
│  "explanation": "The email scores high on personalization as it addresses the recipient by name (Olivia) and    │
│  references their specific background and company (Two Sigma). The tone is professional and matches the         │
│  context of a job opportunity. The email is clear in its intent and provides specific details about the role,   │
│  such as the skills required (Python, statistics, ML, and time-series analysis) and the responsibilities        │
│  (developing predictive trading signals). However, the email could be improved by adding more unique personal   │
│  touches, such as referencing a specific project or achievement of Olivia's, to make it stand out.              │
│  Additionally, the email assumes Olivia might be interested in the role or know someone who is, which could be  │
│  seen as somewhat generic. Overall, the email demonstrates a good balance of personalization, clarity, and      │
│  professionalism, but lacks a bit of creativity and depth to make it truly standout."                           │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()